![uc3m](img/uc3m.jpg)

# The Simplex Algorithm

<a href="http://www.est.uc3m.es/nogales" target="_blank">Javier Nogales</a>

![](img/simplex.png)

## Consider the Airline Revenue Management example (from slides)



![](img/RM_matrix.png)



Problem definition (standard formulation):

In [1]:
import numpy as np

A = np.array([[1,1,1,0,0],
              [1,0,0,1,0],
              [0,1,0,0,1]])

b = np.array([150,75,125])

c = np.array([-400,-150,0,0,0])

# Define problem dimension
(m,n)=A.shape


### Step 0

Start with a natural basic solution: $x=(x_3,x_4,x_5)$

In [2]:
# Initial basis

# Take care: in Python indexes start from 0
# Select 3 columns
iB = np.array([2,3,4])  # I
iN = np.array([0,1])    # J

# Basic and non-basic matrices
B = A[:,iB]
N = A[:,iN]

print(B)

print(N)

[[1 0 0]
 [0 1 0]
 [0 0 1]]
[[1 1]
 [1 0]
 [0 1]]


Compute now the basic solution part ($x_B=B^{-1}b$) and the non-basic one ($x_N=0$)

In [3]:
# Compute the first vertex: x=(x_B | x_N)

xB = np.linalg.solve(B,b)
xN = np.zeros(n-m)

print(xB)

print(xN)

[150.  75. 125.]
[0. 0.]


Is it a vertex (basic feasible solution)?

Join now the basic part with the non-basic one

In [7]:
# The vertex (basic solution) with the componentes in the right order

x = np.concatenate((xN, xB))
index = np.concatenate((iN, iB))
x = x[np.argsort(index)]

print(x)
print(index)

[  0.   0. 150.  75. 125.]
[0 1 2 3 4]


### Step 1

Compute marginal prices (Lagrange multipliers) and reduced costs

In [8]:
## Step 1 ##

lam = np.linalg.solve(B.T,c[iB]) # marginal prices (associated to equality constraints)

sigmaN = c[iN]-np.dot(lam,N) # reduced costs (associated to positivity constraints)

print(lam,sigmaN)

[0. 0. 0.] [-400. -150.]


Do we have the optimal solution?


In [9]:
if np.all(sigmaN>0):
    print("Stop: optimal solution found")

### Step 2

Select the non-basic variable $j$ that is going to enter the basis

In [10]:
## Step 2 ##

j = np.argmin(sigmaN)

# j-th column of N
Nj = N[:,j]

print(Nj)

print(j)

[1 1 0]
0


Compute now the search direction $p$

In [11]:
# search direction

# basic part: solve a linear system of equations
pB = -np.linalg.solve(B, Nj)

# non-basic part: unit vector
pN = np.zeros(n-m)
pN[j]=1

print(pB,pN)

p = np.zeros(n)
p[iB]=pB
p[iN]=pN

print(p)

[-1. -1. -0.] [1. 0.]
[ 1.  0. -1. -1. -0.]


Is the solution unbounded?

### Step 3

Select the basic variable $i$ that is going to enter the non-basic matrix

In [12]:
## Step 3 ##

# find components where basic variables could become negative
index = np.where(pB < 0)

# find possible step lengths to avoid negative basic variables
alpha = - xB[index] / pB[index]

# variable leaving basis
i = index[0][np.argmin(alpha)]

# choose the minimum alpha to avoid negative values
alpha = np.min(alpha)

print(alpha)

print(i)

75.0
1


### Step 4

Update

In [13]:
## Step 4 ##

print(iB,iN)

iBnew=list(iB)
iBnew[i]=iN[j]

iNnew=list(iN)
iNnew[j]=iB[i]

print(iBnew,iNnew)

iB = np.sort(iBnew)
iN = np.sort(iNnew)

print(iB,iN)

[2 3 4] [0 1]
[2, 0, 4] [3, 1]
[0 2 4] [1 3]


Update B and N, and move to the next basic solution

In [14]:
# ADD YOUR CODE HERE
# Most important function
B = A[:,iB]
N = A[:, iN]
X=x+alpha*p

print(x)

[  0.   0. 150.  75. 125.]


Is the new point a vertex (basic feasible solution)?

In [15]:
print(B)

print(N)

print(alpha)

print(p)

print(np.linalg.solve(B,b))
# new solution is a new vertex


[[1 1 0]
 [1 0 0]
 [0 0 1]]
[[1 0]
 [0 1]
 [1 0]]
75.0
[ 1.  0. -1. -1. -0.]
[ 75.  75. 125.]


### Go to Step 1 and repeat

In [ ]:
# Add here the code to make another iteration

